<a href="https://colab.research.google.com/github/gupta799/LLMFinetuning/blob/main/knowledge_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from torch import nn, optim
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Hyperparameters
alpha = 0.5
beta = 0.5
temperature = 2.0
batch_size = 64
learning_rate = 5e-5
num_epochs = 3

In [ ]:
teacher_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
teacher_model.eval()
student_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
student_model.train()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# Move models to device
teacher_model.to(device)
student_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import numpy as np
dataset = load_dataset("glue", "sst2")
train_indices = np.random.choice(len(dataset['train']), size=1000, replace=False)
dataset = dataset['train'].select(train_indices)
print(dataset)
def tokenize_function(example):
    return tokenizer(example['sentence'], padding='max_length', truncation=True, max_length=128)

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 1000
})


In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Set the format to return PyTorch tensors
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


In [ ]:
train_dataset = tokenized_datasets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
ce_loss = nn.CrossEntropyLoss()
kl_div_loss = nn.KLDivLoss(reduction='batchmean')

In [ ]:
optimizer = optim.AdamW(student_model.parameters(), lr=learning_rate)


In [ ]:
for epoch in range(num_epochs):
    total_loss = 0
    student_model.train()  # Ensure student is in training mode
    for batch in train_loader:
        # Prepare inputs
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }

        if 'token_type_ids' in batch and batch['token_type_ids'] is not None:
            inputs['token_type_ids'] = batch['token_type_ids'].to(device)

        labels = batch['labels'].to(device)

        # Forward pass through teacher
        with torch.no_grad():
            teacher_outputs = teacher_model(**inputs)
            teacher_logits = teacher_outputs.logits

        # Forward pass through student
        student_outputs = student_model(**inputs)
        student_logits = student_outputs.logits

        # Compute losses
        loss_ce = ce_loss(student_logits, labels)
        loss_kl = kl_div_loss(
            nn.functional.log_softmax(student_logits / temperature, dim=-1),
            nn.functional.softmax(teacher_logits / temperature, dim=-1)
        ) * (temperature ** 2)

        loss = alpha * loss_ce + beta * loss_kl

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print(f"Batch Loss: {loss.item()}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Average Loss: {avg_loss:.4f}")

Batch Loss: 0.26357734203338623
Batch Loss: 0.25026649236679077
Batch Loss: 0.23788970708847046
Batch Loss: 0.26345211267471313
Batch Loss: 0.25933465361595154
Batch Loss: 0.24187670648097992
Batch Loss: 0.25566786527633667
Batch Loss: 0.25461360812187195
Batch Loss: 0.2735750675201416
Batch Loss: 0.25423216819763184
Batch Loss: 0.2533288598060608
Batch Loss: 0.2581234872341156
Batch Loss: 0.2550610601902008
Batch Loss: 0.2509770691394806
Batch Loss: 0.2342389076948166
Batch Loss: 0.2466750144958496
Epoch 1/3 - Average Loss: 0.2533
Batch Loss: 0.24015331268310547
Batch Loss: 0.23438924551010132
Batch Loss: 0.2410677969455719
Batch Loss: 0.2507525682449341
Batch Loss: 0.23290106654167175
Batch Loss: 0.23582205176353455
Batch Loss: 0.24088548123836517
Batch Loss: 0.22644834220409393
Batch Loss: 0.23237371444702148
Batch Loss: 0.2396470606327057
Batch Loss: 0.23573614656925201
Batch Loss: 0.23544642329216003
Batch Loss: 0.2353137582540512
Batch Loss: 0.24009770154953003
Batch Loss: 0.2595